In [1]:
import gym
import numpy as np

from gym.wrappers import AtariPreprocessing
gym.new_step_api=True
env = gym.make('Pong-v0')

H = 200 # number of hidden layer neurons
D = 80 * 80 # input dimensionality: 80x80 grid
model = {}
model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
model['W2'] = np.random.randn(H) / np.sqrt(H)
# hyperparameters
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-2
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
grad_buffer = { k : np.zeros_like(v) for k,v in model.items() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.items() } # rmsprop memory

def sigmoid(x): 
  return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
  I=np.asarray(I)
  I = I[35:195] # crop
  I = I[::2,::2,0] # downsample by factor of 2
  
  I[I == 144] = 0 # erase background (background type 1)
  
  I[I == 109] = 0 # erase background (background type 2)
  
  I[I != 0] = 1 # everything else (paddles, ball) just set to 1
  return I.astype(float).ravel()

def discount_rewards(r):
  discounted_r = np.zeros_like(r)
  running_add = 0
  for t in reversed(range(0, r.size)):
    if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
    running_add = running_add * gamma + r[t]
    discounted_r[t] = running_add
  return discounted_r

def policy_forward(x):
  h = np.dot(model['W1'], x)
  h[h<0] = 0 # ReLU nonlinearity
  logp = np.dot(model['W2'], h)
  p = sigmoid(logp)
  return p, h # return probability of taking action 2, and hidden state

def policy_backward(epx, eph, epdlogp):
  """ backward pass. (eph is array of intermediate hidden states) """
  dW2 = np.dot(eph.T, epdlogp).ravel()
  dh = np.outer(epdlogp, model['W2'])
  dh[eph <= 0] = 0 # backpro prelu
  dW1 = np.dot(dh.T, epx)
  return {'W1':dW1, 'W2':dW2}

def model_step(model, observation, prev_x):

  cur_x = prepro(observation)
  x = cur_x - prev_x if prev_x is not None else np.zeros(D)
  prev_x = cur_x
  
  # forward the policy network and sample an action from the returned probability
  aprob, _ = policy_forward(x)
  action = 2 if aprob >= 0.5 else 3 # roll the dice!
  
  return action, prev_x

def play_game(env, model):
  observation = env.reset()

  frames = []
  cumulated_reward = 0

  prev_x = None # used in computing the difference frame

  for t in range(1000):
      frames.append(env.render(mode = 'rgb_array'))
      action, prev_x = model_step(model, observation, prev_x)
      observation, reward, done, info = env.step(action)
      cumulated_reward += reward
      if done:
          print("Episode finished after {} timesteps, accumulated reward = {}".format(t+1, cumulated_reward))
          break
  print("Episode finished without success, accumulated reward = {}".format(cumulated_reward))
  env.close()
  display_frames_as_gif(frames)

def train_model(env, model, total_episodes = 100):
  hist = []
  observation = env.reset()

  prev_x = None # used in computing the difference frame
  xs,hs,dlogps,drs = [],[],[],[]
  running_reward = None
  reward_sum = 0
  episode_number = 0

  while True:
  
    cur_x = prepro(observation)
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x

    # forward the policy network and sample an action from the returned probability
    aprob, h = policy_forward(x)
    action = 2 if np.random.uniform() < aprob else 3 # roll the dice!

    # record various intermediates (needed later for backprop)
    xs.append(x) # observation
    hs.append(h) # hidden state
    y = 1 if action == 2 else 0 # a "fake label"
    dlogps.append(y - aprob) # grad that encourages the action that was taken to be taken (see http://cs231n.github.io/neural-networks-2/#losses if confused)

    # step the environment and get new measurements
    observation, reward, done, info = env.step(action)
    reward_sum += reward

    drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

    if done: # an episode finished
      episode_number += 1

      # stack together all inputs, hidden states, action gradients, and rewards for this episode
      epx = np.vstack(xs)
      eph = np.vstack(hs)
      epdlogp = np.vstack(dlogps)
      epr = np.vstack(drs)
      xs,hs,dlogps,drs = [],[],[],[] # reset array memory

      # compute the discounted reward backwards through time
      discounted_epr = discount_rewards(epr)
      # standardize the rewards to be unit normal (helps control the gradient estimator variance)
      discounted_epr -= np.mean(discounted_epr)
      discounted_epr /= np.std(discounted_epr)

      epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
      grad = policy_backward(epx, eph, epdlogp)
      for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

      # perform rmsprop parameter update every batch_size episodes
      if episode_number % batch_size == 0:
        for k,v in model.items():
          g = grad_buffer[k] # gradient
          rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
          model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
          grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

      # boring book-keeping
      running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
      hist.append((episode_number, reward_sum, running_reward))
      print ('episode %f, reward total was %f. running mean: %f' % (episode_number, reward_sum, running_reward))
      reward_sum = 0
      observation = env.reset() # reset env
      prev_x = None
      if episode_number == total_episodes: 
        return hist

    # if reward != 0: # Pong has either +1 or -1 reward exactly when game ends.
    #   print('ep {}: game finished, reward: {}'.format(episode_number, reward) + ('' if reward == -1 else ' !!!!!!!!'))

C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\envs\registration.py:592: UserWarning: WARN: The environment Pong-v0 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [2]:
%time hist1 = train_model(env, model, total_episodes=5500)

C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\utils\passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


episode 1.000000, reward total was -20.000000. running mean: -20.000000
episode 2.000000, reward total was -20.000000. running mean: -20.000000
episode 3.000000, reward total was -21.000000. running mean: -20.010000
episode 4.000000, reward total was -21.000000. running mean: -20.019900
episode 5.000000, reward total was -21.000000. running mean: -20.029701
episode 6.000000, reward total was -20.000000. running mean: -20.029404
episode 7.000000, reward total was -20.000000. running mean: -20.029110
episode 8.000000, reward total was -20.000000. running mean: -20.028819
episode 9.000000, reward total was -21.000000. running mean: -20.038531
episode 10.000000, reward total was -21.000000. running mean: -20.048145
episode 11.000000, reward total was -21.000000. running mean: -20.057664
episode 12.000000, reward total was -21.000000. running mean: -20.067087
episode 13.000000, reward total was -21.000000. running mean: -20.076416
episode 14.000000, reward total was -21.000000. running mean

episode 114.000000, reward total was -19.000000. running mean: -20.581575
episode 115.000000, reward total was -21.000000. running mean: -20.585759
episode 116.000000, reward total was -21.000000. running mean: -20.589902
episode 117.000000, reward total was -20.000000. running mean: -20.584003
episode 118.000000, reward total was -21.000000. running mean: -20.588163
episode 119.000000, reward total was -21.000000. running mean: -20.592281
episode 120.000000, reward total was -20.000000. running mean: -20.586358
episode 121.000000, reward total was -21.000000. running mean: -20.590495
episode 122.000000, reward total was -21.000000. running mean: -20.594590
episode 123.000000, reward total was -21.000000. running mean: -20.598644
episode 124.000000, reward total was -21.000000. running mean: -20.602657
episode 125.000000, reward total was -21.000000. running mean: -20.606631
episode 126.000000, reward total was -21.000000. running mean: -20.610564
episode 127.000000, reward total was -

episode 225.000000, reward total was -21.000000. running mean: -20.779526
episode 226.000000, reward total was -21.000000. running mean: -20.781731
episode 227.000000, reward total was -21.000000. running mean: -20.783914
episode 228.000000, reward total was -21.000000. running mean: -20.786075
episode 229.000000, reward total was -21.000000. running mean: -20.788214
episode 230.000000, reward total was -21.000000. running mean: -20.790332
episode 231.000000, reward total was -21.000000. running mean: -20.792428
episode 232.000000, reward total was -21.000000. running mean: -20.794504
episode 233.000000, reward total was -21.000000. running mean: -20.796559
episode 234.000000, reward total was -21.000000. running mean: -20.798593
episode 235.000000, reward total was -21.000000. running mean: -20.800608
episode 236.000000, reward total was -21.000000. running mean: -20.802601
episode 237.000000, reward total was -21.000000. running mean: -20.804575
episode 238.000000, reward total was -

episode 336.000000, reward total was -21.000000. running mean: -20.915595
episode 337.000000, reward total was -21.000000. running mean: -20.916439
episode 338.000000, reward total was -21.000000. running mean: -20.917275
episode 339.000000, reward total was -21.000000. running mean: -20.918102
episode 340.000000, reward total was -21.000000. running mean: -20.918921
episode 341.000000, reward total was -21.000000. running mean: -20.919732
episode 342.000000, reward total was -21.000000. running mean: -20.920535
episode 343.000000, reward total was -21.000000. running mean: -20.921329
episode 344.000000, reward total was -21.000000. running mean: -20.922116
episode 345.000000, reward total was -21.000000. running mean: -20.922895
episode 346.000000, reward total was -21.000000. running mean: -20.923666
episode 347.000000, reward total was -21.000000. running mean: -20.924429
episode 348.000000, reward total was -21.000000. running mean: -20.925185
episode 349.000000, reward total was -

...

episode 4962.000000, reward total was -21.000000. running mean: -20.688832
episode 4963.000000, reward total was -21.000000. running mean: -20.691943
episode 4964.000000, reward total was -21.000000. running mean: -20.695024
episode 4965.000000, reward total was -21.000000. running mean: -20.698074
episode 4966.000000, reward total was -21.000000. running mean: -20.701093
episode 4967.000000, reward total was -21.000000. running mean: -20.704082
episode 4968.000000, reward total was -21.000000. running mean: -20.707041
episode 4969.000000, reward total was -21.000000. running mean: -20.709971
episode 4970.000000, reward total was -21.000000. running mean: -20.712871
episode 4971.000000, reward total was -21.000000. running mean: -20.715742
episode 4972.000000, reward total was -20.000000. running mean: -20.708585
episode 4973.000000, reward total was -21.000000. running mean: -20.711499
episode 4974.000000, reward total was -20.000000. running mean: -20.704384
episode 4975.000000, rewa

episode 5072.000000, reward total was -21.000000. running mean: -20.651591
episode 5073.000000, reward total was -21.000000. running mean: -20.655075
episode 5074.000000, reward total was -20.000000. running mean: -20.648525
episode 5075.000000, reward total was -21.000000. running mean: -20.652039
episode 5076.000000, reward total was -20.000000. running mean: -20.645519
episode 5077.000000, reward total was -21.000000. running mean: -20.649064
episode 5078.000000, reward total was -21.000000. running mean: -20.652573
episode 5079.000000, reward total was -20.000000. running mean: -20.646047
episode 5080.000000, reward total was -21.000000. running mean: -20.649587
episode 5081.000000, reward total was -20.000000. running mean: -20.643091
episode 5082.000000, reward total was -21.000000. running mean: -20.646660
episode 5083.000000, reward total was -21.000000. running mean: -20.650193
episode 5084.000000, reward total was -20.000000. running mean: -20.643692
episode 5085.000000, rewa

episode 5182.000000, reward total was -21.000000. running mean: -20.602006
episode 5183.000000, reward total was -21.000000. running mean: -20.605986
episode 5184.000000, reward total was -20.000000. running mean: -20.599926
episode 5185.000000, reward total was -20.000000. running mean: -20.593927
episode 5186.000000, reward total was -21.000000. running mean: -20.597988
episode 5187.000000, reward total was -21.000000. running mean: -20.602008
episode 5188.000000, reward total was -21.000000. running mean: -20.605988
episode 5189.000000, reward total was -21.000000. running mean: -20.609928
episode 5190.000000, reward total was -21.000000. running mean: -20.613829
episode 5191.000000, reward total was -21.000000. running mean: -20.617691
episode 5192.000000, reward total was -20.000000. running mean: -20.611514
episode 5193.000000, reward total was -20.000000. running mean: -20.605398
episode 5194.000000, reward total was -21.000000. running mean: -20.609345
episode 5195.000000, rewa

episode 5292.000000, reward total was -21.000000. running mean: -20.658941
episode 5293.000000, reward total was -21.000000. running mean: -20.662351
episode 5294.000000, reward total was -20.000000. running mean: -20.655728
episode 5295.000000, reward total was -21.000000. running mean: -20.659171
episode 5296.000000, reward total was -21.000000. running mean: -20.662579
episode 5297.000000, reward total was -20.000000. running mean: -20.655953
episode 5298.000000, reward total was -21.000000. running mean: -20.659394
episode 5299.000000, reward total was -21.000000. running mean: -20.662800
episode 5300.000000, reward total was -20.000000. running mean: -20.656172
episode 5301.000000, reward total was -21.000000. running mean: -20.659610
episode 5302.000000, reward total was -21.000000. running mean: -20.663014
episode 5303.000000, reward total was -21.000000. running mean: -20.666384
episode 5304.000000, reward total was -20.000000. running mean: -20.659720
episode 5305.000000, rewa

episode 5402.000000, reward total was -21.000000. running mean: -20.713018
episode 5403.000000, reward total was -21.000000. running mean: -20.715888
episode 5404.000000, reward total was -21.000000. running mean: -20.718729
episode 5405.000000, reward total was -20.000000. running mean: -20.711542
episode 5406.000000, reward total was -20.000000. running mean: -20.704426
episode 5407.000000, reward total was -21.000000. running mean: -20.707382
episode 5408.000000, reward total was -21.000000. running mean: -20.710308
episode 5409.000000, reward total was -20.000000. running mean: -20.703205
episode 5410.000000, reward total was -21.000000. running mean: -20.706173
episode 5411.000000, reward total was -21.000000. running mean: -20.709111
episode 5412.000000, reward total was -21.000000. running mean: -20.712020
episode 5413.000000, reward total was -21.000000. running mean: -20.714900
episode 5414.000000, reward total was -21.000000. running mean: -20.717751
episode 5415.000000, rewa